In [165]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Union
import os

In [166]:
M = 1.989e30 #kg
G = 6.6741e-11 # m^3/kg/s^2
AU = 149597870700 #m

In [167]:
x0 = np.array([0, 0.586]) * AU # m
v0 = np.array([54600, 0]) # m/s


In [168]:
def a(r: np.ndarray):
    return -G * M * r / np.power(r.T.dot(r), 3/2)
    

In [169]:
class Euler:
    def __init__(self, tmax: int, dt: float):
        self.x = None
        self.v = None
        self.t = None
        self.dt = None
        self.update_params(tmax, dt)

    def update_params(self, tmax: int, dt: float):
        num = int(tmax//dt)
        self.x = np.zeros((num, 2))
        self.v = np.zeros((num, 2))
        self.t = np.linspace(0, tmax, num=num)
        self.dt = dt

    def get_next_step(self, i: int):
        last_x = self.x[i]
        last_v = self.v[i]
        new_x = last_x + last_v * self.dt
        new_v = last_v + a(last_x) * self.dt
        return new_x, new_v
    
    def set_new_values(self, x: np.ndarray, v: np.ndarray, i):
        self.x[i] = x
        self.v[i] = v

    def get_new_values(self, i: int):
        return self.x[i], self.v[i]

    def calculate(self):
        for i in range(len(self.t)):
            if i == 0:
                self.set_new_values(x0, v0, 0)
                continue
            x, y = self.get_next_step(i-1)
            self.set_new_values(x, y, i)

In [194]:
class RK4:
    def __init__(self, tmax: int, dt: float):
        self.values = None
        self.t = None
        self.dt = None
        self.k = None
        self.update_params(tmax, dt)

    def update_params(self, tmax: int, dt: float):
        num = int(tmax//dt)
        self.values = np.zeros((num, 6))  # x, y, vx, vy, ax, ay
        self.t = np.linspace(0, tmax, num=num)
        self.dt = dt

    def get_r(self, i: int) -> np.ndarray:
        return self.values[i, 0:2]


    def get_v(self, i: int) -> np.ndarray:
        return self.values[i, 2:4]

    def set_values(self, i: int, values: np.ndarray):
        self.values[i] = values  .ravel()  

    def get_u(self, i: int) -> np.ndarray:
        return self.values[i, 0:4]

    def get_f(self, i: int) -> np.ndarray:
        return self.values[i, 2:]

    def get_k(self, i: int) -> np.ndarray:
        k1 = self.calculate_k1(i)
        k2 = self.calculate_k2(k1, i)
        k3 = self.calculate_k3(k2, i)
        k4 = self.calculate_k4(k3, i)
        return np.array([k1, k2, k3, k4])

    def calculate_k1(self, i: int) -> np.ndarray:
        return self.get_f(i).T

    def calculate_k2(self, k1: float, i: int) -> np.ndarray:
        return np.concatenate((
            self.get_v(i) + k1[2:] * self.dt / 2,
            a(self.get_r(i) + k1[:2] * self.dt / 2)
        )).T

    def calculate_k3(self, k2: float, i: int) -> np.ndarray:
        return np.concatenate((
            self.get_v(i) + k2[2:] * self.dt / 2,
            a(self.get_r(i) + k2[:2] * self.dt / 2)
        )).T

    def calculate_k4(self, k3: float, i: int) -> np.ndarray:
        return np.concatenate((
            self.get_v(i) + k3[2:] * self.dt,
            a(self.get_r(i) + k3[:2] * self.dt)
        )).T

    def get_next_step(self, i: int) -> np.ndarray:
        k = self.get_k(i)

        vec = np.array([1, 2, 2, 1]).T
        initial = self.values[i,:]
        values = initial + self.dt / 6 * np.concatenate((vec.T.dot(k).T), a(initial[:2]))
        
        return values

    def calculate_steps(self):
        length = len(self.t)
        for i in range(length):
            if length % 2000 == 0:
                print(f"{100*i/length}%")
            if i == 0:
                initial_values = np.concatenate((x0, v0, a(x0)))
                self.set_values(0, initial_values)
                continue
            values = self.get_next_step(i-1)
            self.set_values(i, values)


In [195]:
class Plotter:
    def __init__(self, solver: Euler, dir: str):
        self.solver = solver
        self.dir = dir
        if not os.path.exists(dir):
            os.makedirs(dir)
        

    def plot(self, filename: str):
        fig, [ax1, ax2] = plt.subplots(2, 1)
        ax1.plot(self.solver.x[:, 0], self.solver.x[:, 1])
        ax1.set_xlabel('x')
        ax1.set_ylabel('y')
        ax2.plot(self.solver.t, self.solver.x[:, 1])
        ax2.set_xlabel('t')
        ax2.set_ylabel('y')
        fig.savefig(f"{self.dir}/{filename}.png")



In [196]:
class Handler:
    def __init__(self):
        self.dir = "output"
    
    def euler_fixed_dt(self):
        solver = Euler(5000000000, 20000)
        solver.calculate()
        plotter = Plotter(solver, self.dir)
        plotter.plot("euler_fixed")

    def rk4_fixed_dt(self):
        solver = RK4(10000000000, 2000)
        solver.calculate_steps()
        plotter = Plotter(solver, self.dir)
        plotter.plot("rk4_fixed")

In [197]:
h = Handler()
h.rk4_fixed_dt()

0.0%
2e-05%
4e-05%
6e-05%
8e-05%
0.0001%
0.00012%
0.00014%
0.00016%
0.00018%
0.0002%
0.00022%
0.00024%
0.00026%
0.00028%
0.0003%
0.00032%
0.00034%
0.00036%
0.00038%
0.0004%
0.00042%
0.00044%
0.00046%
0.00048%
0.0005%
0.00052%
0.00054%
0.00056%
0.00058%
0.0006%
0.00062%
0.00064%
0.00066%
0.00068%
0.0007%
0.00072%
0.00074%
0.00076%
0.00078%
0.0008%
0.00082%
0.00084%
0.00086%
0.00088%
0.0009%
0.00092%
0.00094%
0.00096%
0.00098%
0.001%
0.00102%
0.00104%
0.00106%
0.00108%
0.0011%
0.00112%
0.00114%
0.00116%
0.00118%
0.0012%
0.00122%
0.00124%
0.00126%
0.00128%
0.0013%
0.00132%
0.00134%
0.00136%
0.00138%
0.0014%
0.00142%
0.00144%
0.00146%
0.00148%
0.0015%
0.00152%
0.00154%
0.00156%
0.00158%
0.0016%
0.00162%
0.00164%
0.00166%
0.00168%
0.0017%
0.00172%
0.00174%
0.00176%
0.00178%
0.0018%
0.00182%
0.00184%
0.00186%
0.00188%
0.0019%
0.00192%
0.00194%
0.00196%
0.00198%
0.002%
0.00202%
0.00204%
0.00206%
0.00208%
0.0021%
0.00212%
0.00214%
0.00216%
0.00218%
0.0022%
0.00222%
0.00224%
0.00226%
0.00228%
0

KeyboardInterrupt: 